# Summary of This Notebook
In this notebook, our main goal is to create a database to store all the information we crawled from Last.fm API and Seatgeek API. Since Seatgeek API has a rate limitation and it's concert data is time variate, we need to use cron to make the code run in the background. All of the following codes are inspired by Prof. Panos Ipeirotis in NYU Stern. One of our group member took his data science introduction class in Fall 2017.

In [1]:
import pandas as pd
import requests
from datetime import datetime
import MySQLdb as mdb
import sys
import pickle
import pytz as tz

# 1. Connect to Database

In [2]:
con = mdb.connect(host = 'your IP', 
                  user = 'root',
                  database = 'My_Project',
                  passwd = '*****', 
                  charset='utf8', use_unicode=True);

In [3]:
# Run a query to create a database that will hold the data
db_name = 'ML_Project_Final'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
cursor = con.cursor()
cursor.execute(create_db_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: Warning: (1007, "Can't create database 'ML_Project_Final'; database exists")
  import sys


# 2. Create Artist Info Table from Last.fm
Here, we created a table to store artists' information from Last.fm. In addition, we also created a another file to store all the artist names, which will be used as keys to crawl concert information from Seatgeek.

In [4]:
api_root_url = "http://ws.audioscrobbler.com/2.0/"
url_params = "?method=chart.gettopartists&api_key=d31531b394866cb9db67526b455f274b&format=json&limit=1000&page="

url = api_root_url + url_params
data = requests.get(url).json()


In [5]:
def get_artist_lastfm(url):
    artist_list = []
    for i in range(1,11):
        url2 = url + str(i)
        data = requests.get(url2).json()
        data = data['artists']['artist']
        for j in range(len(data)):
            dict_artist = {}
            dict_artist['name'] = data[j]['name']
            dict_artist['playcount'] = int(data[j]['playcount'])
            dict_artist['image'] = data[j]['image'][-1]['#text'] #-1 gets the largest size
            artist_list.append(dict_artist)
    return artist_list

In [6]:
artist_names = get_artist_lastfm(url)
print('Number of Artists: ', len(artist_names))

Number of Artists:  9050


In [7]:
artist_names_df = pd.DataFrame(artist_names)
artist_names_df.head()

,image,name,playcount
0,https://lastfm-img2.akamaized.net/i/u/300x300/...,Maribou State,2160319
1,https://lastfm-img2.akamaized.net/i/u/300x300/...,Dead Can Dance,28439014
2,https://lastfm-img2.akamaized.net/i/u/300x300/...,The Stooges,15966864
3,https://lastfm-img2.akamaized.net/i/u/300x300/...,Tee Grizzley,684626
4,https://lastfm-img2.akamaized.net/i/u/300x300/...,Novo Amor,1736192


In [8]:
artist_names_df["name"] = artist_names_df["name"].apply(lambda x: x.replace(" ", "-"))
artist_names_df["name"] = artist_names_df["name"].apply(lambda x: x.lower())
artist_name_list = artist_names_df["name"]
artist_name_list[:5]

0     maribou-state
1    dead-can-dance
2       the-stooges
3      tee-grizzley
4         novo-amor
Name: name, dtype: object

In [9]:
# Create the file of aritist names, which will be used as search keys.
file_Name = "seatgeek_artist_name_list"
fileObject = open(file_Name,'wb') 

pickle.dump(artist_name_list,fileObject)
fileObject.close()

In [10]:
cursor = con.cursor()
table_name = 'lastfm'

create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (artist varchar(250), 
                                playcount int,
                                image_url varchar(250),
                                PRIMARY KEY(artist)
                                )'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: Warning: (1050, "Table 'lastfm' already exists")
  # Remove the CWD from sys.path while we load stuff.


In [11]:
#Insert artist data
cursor = con.cursor()
table_name = 'lastfm'

query_template = '''INSERT IGNORE INTO {db}.{table}(artist, 
                                            playcount,
                                            image_url) 
                                            VALUES (%s, %s, %s)'''.format(db=db_name, table=table_name)

cursor = con.cursor()

artist_names = artist_names

for i in range(len(artist_names)):
    artist = artist_names[i]['name']
    playcount = artist_names[i]['playcount']
    image_url = artist_names[i]['image']
    
    query_parameters = (artist, playcount, image_url)
    cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Maribou State' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Dead Can Dance' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'The Stooges' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Tee Grizzley' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Novo Amor' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'SunMi' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Vulfpeck' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Cults' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Panda Bear' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'You Me at Six' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Erykah Badu' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Porter Robinson' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'The Postal Service' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Violent Femmes' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Wa

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Maluma' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Coin' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Creed' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'TV Girl' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Ikon' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Ride' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Charlotte Lawrence' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Mark K

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Luchs' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Bob Moses' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Sir Sly' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Thelonious Monk' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Mayday Parade' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Mr. Probz' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Nina Nesbitt' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "D

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Maurice Ravel' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Camille Saint-Saëns' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Harrison Storm' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Redbone' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Sigma' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Blue Swede' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Elvis Costello & The Attractions' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_laun

...

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Vagabon' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Horse Head' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Kuranes' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'the GazettE' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'F.T Island' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Casey' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'The Six Parts Seven' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Du

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Panda' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'O.C.' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Night Beats' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Olexesh' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'King Woman' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'BOWNIK' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Tore Bojsten' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'nbsplv' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Dark Cat' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Eric Hutchinson' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Adrenaline Mob' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Sneaky Sound System' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Necrophobic' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Tones on Tail' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Wa

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Caloncho' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'vianney' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Little Jesus' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'LOLO' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Marshmello & Anne-Marie' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'drkmnd' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Darkstar' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Dup

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Penomeco' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Knowsum' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Amnesty' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'The Concretes' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Terranova' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Lamp' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entry 'Biohazard' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: Warning: (1062, "Duplicate entr

# 3. Create Concert Info Table from Seatgeek

In [12]:
cursor = con.cursor()
table_name = 'seatgeek_artists'

create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (artist varchar(250), 
                                popularity varchar(250),
                                genres varchar(250),
                                PRIMARY KEY(artist)
                                )'''.format(db= db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: Warning: (1050, "Table 'seatgeek_artists' already exists")
  # Remove the CWD from sys.path while we load stuff.


In [13]:
#A function that shifts the list each time to solve the crawling limitation problem 
def shift(pickled_list, n):
    return pickled_list[n:] + pickled_list[:n]

In [14]:
#Open pickled file containing the artist names
fileObject = open("seatgeek_artist_name_list",'rb') 
old_pickled_list = list(pickle.load(fileObject))


In [15]:
list_20 = list(old_pickled_list[0:20])
list_20

['maribou-state',
 'dead-can-dance',
 'the-stooges',
 'tee-grizzley',
 'novo-amor',
 'sunmi',
 'vulfpeck',
 'sophie',
 'kim-petras',
 'don-diablo',
 'idles',
 'cnco',
 'fugazi',
 'alec-benjamin',
 'cashmere-cat',
 'destroyer',
 'sixpence-none-the-richer',
 'death-from-above-1979',
 'paolo-nutini',
 'the-pretty-reckless']

In [16]:
# #Insert artist data
# cursor = con.cursor()
# table_name = 'seatgeek_artists'

# query_template = '''INSERT IGNORE INTO {db}.{table}(artist, 
#                                             popularity,
#                                             genres) 
#                                             VALUES (%s, %s, %s)'''.format(db=db_name, table=table_name)

# cursor = con.cursor()


# for name in list_20:
#     dict1 = get_artist_data_seatgeek(name)
#     if dict1 != None:
#         artist = name
#         popularity = dict1['popularity']
#         genres = dict1['genres']
    
#         query_parameters = (artist, popularity, genres)
#         cursor.execute(query_template, query_parameters)

# con.commit()
# cursor.close()

In [17]:
#Function for Table 1. This gets a list of upcoming events and related information
def get_event_data_seatgeek(artist):
    ''' Fetches upcoming concert/events information'''
    url = "https://api.seatgeek.com/2/events?performers.slug=%s&client_id=MTM2Mjg5NjJ8MTU0MDA5MTc4Ny4wMQ"%(artist)
    try:
        if requests.get(url).json()['events'] != None:
            events = requests.get(url).json()['events']
            events_list = []
            for i in range(len(events)-1):
                dict_event = {}
                dict_event['artist'] = artist
                dict_event['title'] = events[i]['short_title'] #title
                dict_event['concert_date'] = datetime.strptime(events[i]['datetime_utc'][0:10], '%Y-%m-%d') #datetime of performance
                dict_event['address'] = events[0]['venue']['address'] + ', ' + events[0]['venue']['extended_address'] #address
                dict_event['venue'] = events[0]['venue']['name'].lower().replace(" ", "-")#venue name
                dict_event['lat'] = events[i]['venue']['location']['lat'] #latitude
                dict_event['lon'] = events[i]['venue']['location']['lon'] #longitude
                dict_event['average_price'] = events[i]['stats']['average_price'] #average price
                dict_event['highest_price'] = events[i]['stats']['highest_price'] #highest price
                dict_event['lowest_price'] = events[i]['stats']['lowest_price'] #lowest price
                dict_event['good_deals'] = events[i]['stats']['lowest_price_good_deals'] #best deal
                events_list.append(dict_event)
            return events_list
        else:
            None
    except:
        None

In [18]:
#For loop to get all the results in one list
input_list = []
for name in list_20:
    input_list.append(get_event_data_seatgeek(name))


In [19]:
#Get rid of artists that have no upcoming concerts, or are not on the Seatgeek database
input_list = [x for x in input_list if x != None] 
input_list = [x for x in input_list if x != []]
input_list

[[{'address': '1115 U Street NW, Washington, DC 20009',
   'artist': 'maribou-state',
   'average_price': 131.34483,
   'concert_date': datetime.datetime(2019, 2, 13, 0, 0),
   'good_deals': None,
   'highest_price': 562,
   'lat': 38.9172,
   'lon': -77.0278,
   'lowest_price': 44,
   'title': 'Maribou State (18+)',
   'venue': 'u-street-music-hall'},
  {'address': '1115 U Street NW, Washington, DC 20009',
   'artist': 'maribou-state',
   'average_price': 117,
   'concert_date': datetime.datetime(2019, 2, 14, 0, 0),
   'good_deals': None,
   'highest_price': 562,
   'lat': 40.7095,
   'lon': -73.9233,
   'lowest_price': 56,
   'title': 'Maribou State (16+)',
   'venue': 'u-street-music-hall'},
  {'address': '1115 U Street NW, Washington, DC 20009',
   'artist': 'maribou-state',
   'average_price': 84.828575,
   'concert_date': datetime.datetime(2019, 2, 15, 0, 0),
   'good_deals': None,
   'highest_price': 562,
   'lat': 42.3529,
   'lon': -71.1326,
   'lowest_price': 31,
   'title': 

In [20]:
#Create a table for SeatGeek Concerts
cursor = con.cursor()
table_name = 'seatgeek_concerts'

create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (artist varchar(250), 
                                concert_date datetime,
                                average_price int,
                                highest_price int,
                                lowest_price int,
                                good_deals int,
                                address varchar(250),
                                venue varchar(250),
                                latitude int,
                                longitude int,
                                update_time datetime,
                                PRIMARY KEY(artist, concert_date)
                                )'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: Warning: (1050, "Table 'seatgeek_concerts' already exists")


In [21]:
#Insert concert data
cursor = con.cursor()
table_name = 'seatgeek_concerts'

query_template = '''INSERT IGNORE INTO {db}.{table}(artist, 
                                            concert_date,
                                            average_price,
                                            highest_price,
                                            lowest_price,
                                            good_deals,
                                            address,
                                            venue,
                                            latitude,
                                            longitude,
                                            update_time) 
                                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)

cursor = con.cursor()



for data in input_list:
    if data != None:
        for j in range(len(data)):
            artist = data[j]['artist']
            concert_date = data[j]['concert_date']
            average_price = data[j]['average_price']
            highest_price = data[j]['highest_price']
            lowest_price = data[j]['lowest_price']
            good_deals = data[j]['good_deals']
            address = data[j]['address']
            venue = data[j]['venue']
            latitude = data[j]['lat']
            longitude = data[j]['lon']
            update_time = datetime.now(tz.timezone('America/New_York')).strftime("%Y-%m-%d %H:%M:%S %Z%z")
            
            query_parameters = (artist, concert_date, average_price, highest_price,
                            lowest_price, good_deals, address, venue,
                            latitude, longitude, update_time)
            cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: Warning: (1265, "Data truncated for column 'update_time' at row 1")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: Warning: (1062, "Duplicate entry 'maribou-state-2019-02-13 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: Warning: (1062, "Duplicate entry 'maribou-state-2019-02-14 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: Warning: (1062, "Duplicate entry 'maribou-state-2019-02-15 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: Warning: (1062, "Duplicate entry 'maribou-state-2019-02-17 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: Warning: (1062, "Duplicate entry 'maribou-state-2019-02-19 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: Warning: (1062, "Duplicate entry

# 4. Create Seatgeek Artists Table

In [22]:
#Functions for Table 2. This gets SeatGeek information on each artist.
def get_genres(info_artist):
    ''' Function to get the genres'''
    string = ""
    genres = info_artist['performers'][0]['genres']
    for i in range(len(genres)):
        string += genres[i]['slug'] + "/"
    return string

def get_artist_data_seatgeek(artist):
    ''' Function to get SeatGeek data on each artist. Quite limited but could be useful'''
    url = "https://api.seatgeek.com/2/performers?slug=" + artist + "&client_id=MTM2Mjg5NjJ8MTU0MDA5MTc4Ny4wMQ"
    try:
        if requests.get(url).json() != None:
            info_artist = requests.get(url).json()
            infolist = {}
            infolist['name'] = artist
            infolist['popularity'] = info_artist['performers'][0]['score']
            infolist['genres'] = get_genres(info_artist)
        return infolist
    except:
        None

In [23]:
#Create a table for SeatGeek Artists
cursor = con.cursor()
table_name = 'seatgeek_artists'

create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (artist varchar(250), 
                                popularity varchar(250),
                                genres varchar(250),
                                PRIMARY KEY(artist)
                                )'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: Warning: (1050, "Table 'seatgeek_artists' already exists")
  # This is added back by InteractiveShellApp.init_path()


In [24]:
#Insert artist data
cursor = con.cursor()
table_name = 'seatgeek_artists'

query_template = '''INSERT IGNORE INTO {db}.{table}(artist, 
                                            popularity,
                                            genres) 
                                            VALUES (%s, %s, %s)'''.format(db=db_name, table=table_name)

cursor = con.cursor()


for name in list_20:
    dict1 = get_artist_data_seatgeek(name)
    if dict1 != None:
        artist = name
        popularity = dict1['popularity']
        genres = dict1['genres']
    
        query_parameters = (artist, popularity, genres)
        cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: Warning: (1062, "Duplicate entry 'maribou-state' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: Warning: (1062, "Duplicate entry 'dead-can-dance' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: Warning: (1062, "Duplicate entry 'the-stooges' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: Warning: (1062, "Duplicate entry 'tee-grizzley' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: Warning: (1062, "Duplicate entry 'vulfpeck' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: Warning: (1062, "Duplicate entry 'sophie' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: Warning: (1062, "Duplicate entry 'don-diablo' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: Warning: (1062

In [25]:
shifted_list_names = shift(old_pickled_list, 20)
print(len(shifted_list_names))
shifted_list_names[:20]

9050


['herbie-hancock',
 'ryan-adams',
 'the-knocks',
 'max',
 'snakehips',
 'smino',
 'the-avalanches',
 'fitz-and-the-tantrums',
 'killswitch-engage',
 'buffalo-springfield',
 'europe',
 'james-vincent-mcmorrow',
 'jethro-tull',
 'the-game',
 'icona-pop',
 'bush',
 'jessie-ware',
 'alok',
 'michael-kiwanuka',
 'bishop-briggs']

In [26]:
#Updating pickle file with current artists
file_Name = "seatgeek_artist_name_list"
fileObject = open(file_Name,'wb') 

pickle.dump(shifted_list_names,fileObject)
fileObject.close()


We put the Seatgeek Concerts part into a python file and we used crontab to run the file in background every hour every day. Namely, we checked concerts information for 20 artists every hour every day.